<a href="https://colab.research.google.com/github/WashingtonKE/Renewable-Energy-Optimization/blob/main/Whisper_Transcription_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎧 Whisper Transcription – Google Colab Notebook

This notebook lets you transcribe audio (e.g., `.aac`, `.mp3`, `.wav`, `.m4a`) using [OpenAI Whisper](https://github.com/openai/whisper) in Google Colab.

## How to use
1. **Run each cell in order** (Shift+Enter).
2. Upload your audio file when prompted.
3. The transcript will be printed and saved as `transcript.txt` (and optionally `transcript.srt`).

**Tip:** Change `MODEL_SIZE` to `small`, `medium`, or `large` for higher accuracy (needs more GPU memory/time).


In [ ]:
# 🛠️ Step 1: Install Whisper and FFmpeg
!pip -q install openai-whisper
!sudo apt -yq update && !sudo apt -yq install ffmpeg
print("Installation complete.")

In [ ]:
# 🎚️ Step 2: (Optional) Choose model size and language
MODEL_SIZE = "base"  # options: tiny, base, small, medium, large
FORCE_LANGUAGE = None  # e.g., "en" for English, "sw" for Swahili; set to None to auto-detect
print(f"Model size set to: {MODEL_SIZE}")
print(f"Force language: {FORCE_LANGUAGE}")

In [ ]:
# 📤 Step 3: Upload your audio file
from google.colab import files
uploaded = files.upload()
assert uploaded, "No file uploaded. Please run this cell again and select an audio file."
FILENAME = list(uploaded.keys())[0]
print("Uploaded:", FILENAME)

In [ ]:
# 🤖 Step 4: Transcribe with Whisper
import whisper, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = whisper.load_model(MODEL_SIZE, device=device)
kwargs = {}
if FORCE_LANGUAGE:
    kwargs["language"] = FORCE_LANGUAGE

result = model.transcribe(FILENAME, **kwargs)
text = result.get("text", "").strip()
print("\n--- TRANSCRIPTION ---\n")
print(text if text else "(No speech recognized or empty transcript)")

In [ ]:
# 💾 Step 5: Save transcript and subtitles, then download
from google.colab import files

# Save plain text transcript
txt_name = "transcript.txt"
with open(txt_name, "w", encoding="utf-8") as f:
    f.write(result.get("text", "").strip())
print("Saved:", txt_name)

# Create simple SRT subtitles from segments
def to_srt(segments, path):
    def fmt(t):
        h = int(t // 3600)
        m = int((t % 3600) // 60)
        s = int(t % 60)
        ms = int((t - int(t)) * 1000)
        return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"
    with open(path, "w", encoding="utf-8") as srt:
        for i, seg in enumerate(segments, start=1):
            start = fmt(seg.get("start", 0.0))
            end = fmt(seg.get("end", 0.0))
            text = seg.get("text", "").strip()
            srt.write(f"{i}\n{start} --> {end}\n{text}\n\n")

srt_name = "transcript.srt"
to_srt(result.get("segments", []), srt_name)
print("Saved:", srt_name)

# Download files
files.download(txt_name)
files.download(srt_name)